# Energy Mix
**Group 4**  
Jianyi Chen, Jincheng Hong, Tianyu Su, Yicheng Huang

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import pulp as lp
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpStatus

## 2. Simple model

In [31]:
sources = ["solar", "wind", "hydro", "coal", "natural_gas"]
lcoe_cost = {"solar": 86.5, "wind": 106.5, "hydro": 213.0, "coal": 74.0, "natural_gas": 54.0}
pollution_cost = {"solar": 37, "wind": 12, "hydro": 150, "coal": 1000, "natural_gas": 430}
fixed_cost = {"solar": 1516000, "wind": 1389000, "hydro": 6428500, "coal": 4074000, "natural_gas": 1300000}
production_limit = {"solar": 999, "wind": 999, "hydro": 43000.0*1000, "coal": 200*1000, "natural_gas": 500.632*1000}
renewable_indicator = {"solar": 1, "wind": 1, "hydro": 1, "coal": 0, "natural_gas": 0}
consumption_rate = {"solar": 0.90, "wind": 0.85, "hydro": 0.95, "coal": 0.70, "natural_gas": 0.80}

para1 = {
    "LCOE_Cost": [lcoe_cost[source] for source in sources],
    "Pollution_Cost": [pollution_cost[source] for source in sources],
    "Production_Limit": [production_limit[source] for source in sources],
    "Renewable_Indicator": [renewable_indicator[source] for source in sources],
    "Consumption_Rate": [consumption_rate[source] for source in sources]
}

parameters_long = pd.DataFrame(para1, index=sources)

para2 = {
    "source": sources,
    "c": [lcoe_cost[source] for source in sources],
    "p": [pollution_cost[source] for source in sources],
    "l": [production_limit[source] for source in sources],
    "r": [renewable_indicator[source] for source in sources],
}

parameters = pd.DataFrame(para2)

In [32]:
parameters_long

,LCOE_Cost,Pollution_Cost,Production_Limit,Renewable_Indicator,Consumption_Rate
solar,86.5,37,999.0,1,0.90
wind,106.5,12,999.0,1,0.85
hydro,213.0,150,43000000.0,1,0.95
coal,74.0,1000,200000.0,0,0.70
natural_gas,54.0,430,500632.0,0,0.80


In [33]:
parameters

,source,c,p,l,r
0,solar,86.5,37,999.0,1
1,wind,106.5,12,999.0,1
2,hydro,213.0,150,43000000.0,1
3,coal,74.0,1000,200000.0,0
4,natural_gas,54.0,430,500632.0,0


In [41]:
B = 12 * 10**9
D = 25263.2 * 1000
lam = 0.5

sources = parameters['source'].tolist()
c = parameters.set_index('source')['c'].to_dict()
p = parameters.set_index('source')['p'].to_dict()
l = parameters.set_index('source')['l'].to_dict()
r = parameters.set_index('source')['r'].to_dict()

prob = LpProblem("EnergyMix1", LpMinimize)

# Decision variables
x = {s: LpVariable(f"x_{s}", lowBound=0, upBound=l[s], cat='Continuous') for s in sources}

# Objective function
prob += lpSum(p[s] * x[s] for s in sources)

# Constraint1
prob += lpSum(x[s] for s in sources) >= D

# Constraint2
prob += lpSum(c[s] * x[s] for s in sources) <= B

# Constraint3
renewable_numerator = lpSum(x[s] * r[s] for s in sources)
renewable_denominator = lpSum(x[s] for s in sources)
prob += renewable_numerator >= lam * renewable_denominator

# Constraint4
for s in sources:
    prob += x[s] <= l[s]
    prob += x[s] >= 0

prob.solve()

print("Status:", LpStatus[prob.status])

result = parameters.copy()
result["x"] = [x[s].varValue for s in sources]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/0dc4141a72c04712b168b86bac1f76bd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/0dc4141a72c04712b168b86bac1f76bd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 49 RHS
At line 63 BOUNDS
At line 69 ENDATA
Problem MODEL has 13 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1 (-12) rows, 5 (0) columns and 5 (-20) elements
0  Obj 3.6840854e+09 Primal inf 702630.54 (1)
1  Obj 3.7892293e+09
Optimal - objective value 3.7892293e+09
After Postsolve, objective 3.7892293e+09, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 3789229251 - 1 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (W

In [42]:
result

,source,c,p,l,r,x
0,solar,86.5,37,999.0,1,999.0
1,wind,106.5,12,999.0,1,999.0
2,hydro,213.0,150,43000000.0,1,25261202.0
3,coal,74.0,1000,200000.0,0,0.0
4,natural_gas,54.0,430,500632.0,0,0.0


## 3. Simple model

In [33]:
parameters

,source,c,p,l,r
0,solar,86.5,37,999.0,1
1,wind,106.5,12,999.0,1
2,hydro,213.0,150,43000000.0,1
3,coal,74.0,1000,200000.0,0
4,natural_gas,54.0,430,500632.0,0


In [52]:
D = 25263.2 * 1000
E_max = 3.8 * 10**9
lam = 0.5

sources = parameters['source'].tolist()
c = parameters.set_index('source')['c'].to_dict()
p = parameters.set_index('source')['p'].to_dict()
l = parameters.set_index('source')['l'].to_dict()
r = parameters.set_index('source')['r'].to_dict()

prob = LpProblem("EnergyMix2", LpMinimize)

# Decision variables
x = {s: LpVariable(f"x_{s}", lowBound=0, upBound=l[s], cat='Continuous') for s in sources}

# Objective function
prob += lpSum(c[s] * x[s] for s in sources)

# Constraint1
prob += lpSum(x[s] for s in sources) >= D

# Constraint2
prob += lpSum(p[s] * x[s] for s in sources) <= E_max

# Constraint3
renewable_numerator = lpSum(x[s] * r[s] for s in sources)
renewable_denominator = lpSum(x[s] for s in sources)
prob += renewable_numerator >= lam * renewable_denominator

# Constraint4
for s in sources:
    prob += x[s] <= l[s]
    prob += x[s] >= 0

prob.solve()

print("Status:", LpStatus[prob.status])

result = parameters.copy()
result["x"] = [x[s].varValue for s in sources]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/08eef6f8076948d9a95702dfb117d589-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/08eef6f8076948d9a95702dfb117d589-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 18 COLUMNS
At line 49 RHS
At line 63 BOUNDS
At line 69 ENDATA
Problem MODEL has 13 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-11) rows, 5 (0) columns and 10 (-15) elements
0  Obj 5.365087e+09 Primal inf 74998.215 (1)
2  Obj 5.3747126e+09
Optimal - objective value 5.3747126e+09
After Postsolve, objective 5.3747126e+09, infeasibilities - dual 0 (0), primal 8.5367432e-07 (1)
Presolved model was optimal, full model needs cleaning up
Optimal - objective value 5.3747126e+09
Optimal objective 5374712586 - 2 iterations time 0.002,

In [53]:
result

,source,c,p,l,r,x
0,solar,86.5,37,999.0,1,9.990000e+02
1,wind,106.5,12,999.0,1,9.990000e+02
2,hydro,213.0,150,43000000.0,1,2.522274e+07
3,coal,74.0,1000,200000.0,0,0.000000e+00
4,natural_gas,54.0,430,500632.0,0,3.846696e+04


## 4. Optimizing both cost and environmental impact

In [64]:
security = [0.2, 0.1, 0.6, 0.9, 0.85]
parameters["sec"] = security

In [65]:
parameters

,source,c,p,l,r,sec
0,solar,86.5,37,999.0,1,0.20
1,wind,106.5,12,999.0,1,0.10
2,hydro,213.0,150,43000000.0,1,0.60
3,coal,74.0,1000,200000.0,0,0.90
4,natural_gas,54.0,430,500632.0,0,0.85


In [94]:
D = 10000
lam = 0.5
alpha, beta = 0.7, 0.3

sources = parameters['source'].tolist()
c = parameters.set_index('source')['c'].to_dict()
p = parameters.set_index('source')['p'].to_dict()
l = parameters.set_index('source')['l'].to_dict()
r = parameters.set_index('source')['r'].to_dict()

prob = LpProblem("EnergyMix3", LpMinimize)

# Decision variables
x = {s: LpVariable(f"x_{s}", lowBound=0, upBound=l[s], cat='Continuous') for s in sources}

# Objective function
prob += alpha*lpSum(c[s] * x[s] for s in sources) + beta*lpSum(p[s] * x[s] for s in sources)

# Constraint1
prob += lpSum(x[s] for s in sources) >= D

# # Constraint2
# prob += lpSum(c[s] * x[s] for s in sources) <= B

# # Constraint2
# prob += lpSum(p[s] * x[s] for s in sources) <= E_max

# Constraint3
renewable_numerator = lpSum(x[s] * r[s] for s in sources)
renewable_denominator = lpSum(x[s] for s in sources)
prob += renewable_numerator >= lam * renewable_denominator

# Constraint4
for s in sources:
    prob += x[s] <= l[s]
    prob += x[s] >= 0

prob.solve()

print("Status:", LpStatus[prob.status])

result = parameters.copy()
result["x"] = [x[s].varValue for s in sources]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c80929ef0c324c658c1fe2e5a6672584-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c80929ef0c324c658c1fe2e5a6672584-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 43 RHS
At line 56 BOUNDS
At line 62 ENDATA
Problem MODEL has 12 rows, 5 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-10) rows, 5 (0) columns and 10 (-10) elements
0  Obj 0 Primal inf 10000 (1)
2  Obj 1566338.4
Optimal - objective value 1566338.4
After Postsolve, objective 1566338.4, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1566338.4 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.0

In [95]:
result

,source,c,p,l,r,sec,x
0,solar,86.5,37,999.0,1,0.20,999.0
1,wind,106.5,12,999.0,1,0.10,999.0
2,hydro,213.0,150,43000000.0,1,0.60,3002.0
3,coal,74.0,1000,200000.0,0,0.90,0.0
4,natural_gas,54.0,430,500632.0,0,0.85,5000.0


## 5. Optimization involving time dependent energy sources

In [111]:
# sources = ["solar", "wind", "hydro", "coal", "natural_gas"]
# c = {"solar": 86.5, "wind": 106.5, "hydro": 213.0, "coal": 74.0, "natural_gas": 54.0}
# p = {"solar": 37, "wind": 12, "hydro": 150, "coal": 1000, "natural_gas": 430}
# r = {"solar": 1, "wind": 1, "hydro": 1, "coal": 0, "natural_gas": 0}

sources = ["solar", "wind", "hydro", "coal", "natural_gas"]

para = pd.DataFrame({
    "c": [86.5, 106.5, 213.0, 74.0, 54.0],
    "p": [37, 12, 150, 1000, 430],
}, index=sources)

l = pd.DataFrame({
    "l0": [900, 400, 43000000, 2000000, 5000000],
    "l1": [0, 500, 43000000, 2000000, 5000000]
}, index=sources)

Dt = np.array([20000,12000])

para

,c,p
solar,86.5,37
wind,106.5,12
hydro,213.0,150
coal,74.0,1000
natural_gas,54.0,430


In [112]:
l

,l0,l1
solar,900,0
wind,400,500
hydro,43000000,43000000
coal,2000000,2000000
natural_gas,5000000,5000000


In [113]:
Dt

array([20000, 12000])

In [94]:
alpha, beta = 0.7, 0.3

sources = parameters['source'].tolist()
c = parameters.set_index('source')['c'].to_dict()
p = parameters.set_index('source')['p'].to_dict()
l = parameters.set_index('source')['l'].to_dict()
r = parameters.set_index('source')['r'].to_dict()

prob = LpProblem("EnergyMix3", LpMinimize)

# Decision variables
x = {s: LpVariable(f"x_{s}", lowBound=0, upBound=l[s], cat='Continuous') for s in sources}

# Objective function
prob += alpha*lpSum(c[s] * x[s] for s in sources) + beta*lpSum(p[s] * x[s] for s in sources)

# Constraint1
prob += lpSum(x[s] for s in sources) >= D

# # Constraint2
# prob += lpSum(c[s] * x[s] for s in sources) <= B

# # Constraint2
# prob += lpSum(p[s] * x[s] for s in sources) <= E_max

# Constraint3
renewable_numerator = lpSum(x[s] * r[s] for s in sources)
renewable_denominator = lpSum(x[s] for s in sources)
prob += renewable_numerator >= lam * renewable_denominator

# Constraint4
for s in sources:
    prob += x[s] <= l[s]
    prob += x[s] >= 0

prob.solve()

print("Status:", LpStatus[prob.status])

result = parameters.copy()
result["x"] = [x[s].varValue for s in sources]

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c80929ef0c324c658c1fe2e5a6672584-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c80929ef0c324c658c1fe2e5a6672584-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 17 COLUMNS
At line 43 RHS
At line 56 BOUNDS
At line 62 ENDATA
Problem MODEL has 12 rows, 5 columns and 20 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 2 (-10) rows, 5 (0) columns and 10 (-10) elements
0  Obj 0 Primal inf 10000 (1)
2  Obj 1566338.4
Optimal - objective value 1566338.4
After Postsolve, objective 1566338.4, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1566338.4 - 2 iterations time 0.002, Presolve 0.00
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.0

In [95]:
result

,source,c,p,l,r,sec,x
0,solar,86.5,37,999.0,1,0.20,999.0
1,wind,106.5,12,999.0,1,0.10,999.0
2,hydro,213.0,150,43000000.0,1,0.60,3002.0
3,coal,74.0,1000,200000.0,0,0.90,0.0
4,natural_gas,54.0,430,500632.0,0,0.85,5000.0
